In [1]:
import os
import glob
import numpy as np

def load_data(files, parts=range(0, 10)+["x"]):
    dataset_x, dataset_y = [], []

    for part in parts:
        for filepath in glob.iglob(files.format(part)):
            x = np.load(filepath)

            y = [0]*len(parts)
            y[parts.index(part)] = 1

            dataset_x.append(x)
            dataset_y.append(y)

    dataset_x = np.array(dataset_x)
    dataset_y = np.array(dataset_y)

    return dataset_x, dataset_y

dataset_x, dataset_y = load_data(os.path.join("data", "number", "{}", "*.npy"))

print dataset_x.shape

testing_size = 0.33
nb_epoch=8
batch_size=32

(0,)


In [9]:
import keras

from keras.models import Sequential
from keras.layers import Dense

from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

## 3a. MLP Model

In [7]:
mlp_preprocess = lambda dataset: dataset.reshape(dataset.shape[0], dataset.shape[1]*dataset.shape[2]).astype('float32') / 255

dataset = mlp_preprocess(dataset_x)

model = Sequential()
model.add(Dense(dataset.shape[1], input_dim=dataset.shape[1], init='normal', activation='relu'))
model.add(Dense(dataset_y.shape[1], init='normal', activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(dataset, dataset_y, test_size=testing_size)

model.fit(X_train, y_train, validation_data=(X_validation, y_validation), nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

scores = model.evaluate(X_validation, y_validation, verbose=0)
print "Baseline Error of Validation Set: {:.8f}%".format(100-scores[1]*100)

Train on 4217 samples, validate on 2078 samples
Epoch 1/8
4217/4217 [==============================] - 1s - loss: 1.7763 - acc: 0.4589 - val_loss: 1.1553 - val_acc: 0.6453
Epoch 2/8
4217/4217 [==============================] - 0s - loss: 1.0517 - acc: 0.6697 - val_loss: 0.9638 - val_acc: 0.7036
Epoch 3/8
4217/4217 [==============================] - 0s - loss: 0.8661 - acc: 0.7327 - val_loss: 0.9883 - val_acc: 0.6949
Epoch 4/8
4217/4217 [==============================] - 0s - loss: 0.7686 - acc: 0.7641 - val_loss: 0.9200 - val_acc: 0.7300
Epoch 5/8
4217/4217 [==============================] - 1s - loss: 0.6896 - acc: 0.7951 - val_loss: 0.8578 - val_acc: 0.7459
Epoch 6/8
4217/4217 [==============================] - 1s - loss: 0.6125 - acc: 0.8224 - val_loss: 0.6207 - val_acc: 0.8268
Epoch 7/8
4217/4217 [==============================] - 1s - loss: 0.5322 - acc: 0.8449 - val_loss: 0.6556 - val_acc: 0.8037
Epoch 8/8
4217/4217 [==============================] - 0s - loss: 0.4713 - acc: 0.86

## 3b. Simple CNN Model

In [10]:
def cnn_preprocess(dataset):
    h, w = dataset.shape[1], dataset.shape[2]
    dataset = dataset.reshape(dataset.shape[0], h, w, 1).astype('float32') / 255

    return dataset, h, w

dataset, h, w = cnn_preprocess(dataset_x)
print dataset.shape, h, w, dataset_y.shape

shape = (h, w, 1)
output_dim = dataset_y.shape[1]

model = Sequential()
model.add(Convolution2D(32, 5, 5, border_mode='same', input_shape=shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(dataset, dataset_y, test_size=testing_size)

model.fit(X_train, y_train, validation_data=(X_validation, y_validation), nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

scores = model.evaluate(X_validation, y_validation, verbose=0)
print "Baseline Error of Validation Set: {:.8f}%".format(100-scores[1]*100)

(6295, 32, 26, 1) 32 26 (6295, 11)
Train on 4217 samples, validate on 2078 samples
Epoch 1/8
4217/4217 [==============================] - 3s - loss: 1.5003 - acc: 0.5478 - val_loss: 0.8654 - val_acc: 0.7642
Epoch 2/8
4217/4217 [==============================] - 3s - loss: 0.6666 - acc: 0.8165 - val_loss: 0.6290 - val_acc: 0.8272
Epoch 3/8
4217/4217 [==============================] - 2s - loss: 0.4382 - acc: 0.8848 - val_loss: 0.4857 - val_acc: 0.8739
Epoch 4/8
4217/4217 [==============================] - 3s - loss: 0.3160 - acc: 0.9115 - val_loss: 0.4007 - val_acc: 0.8956
Epoch 5/8
4217/4217 [==============================] - 3s - loss: 0.2427 - acc: 0.9334 - val_loss: 0.3593 - val_acc: 0.9047
Epoch 6/8
4217/4217 [==============================] - 3s - loss: 0.1822 - acc: 0.9542 - val_loss: 0.3675 - val_acc: 0.9028
Epoch 7/8
4217/4217 [==============================] - 4s - loss: 0.1544 - acc: 0.9583 - val_loss: 0.3206 - val_acc: 0.9192
Epoch 8/8
4217/4217 [============================

## 3c. Multi-Layer CNN Model

In [11]:
model = Sequential()
model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 4, 4, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_validation, y_validation), nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

scores = model.evaluate(X_validation, y_validation, verbose=0)
print "Baseline Error of Validation Set: {:.8f}%".format(100-scores[1]*100)

Train on 4217 samples, validate on 2078 samples
Epoch 1/8
4217/4217 [==============================] - 2s - loss: 1.6391 - acc: 0.4378 - val_loss: 0.8217 - val_acc: 0.7459
Epoch 2/8
4217/4217 [==============================] - 1s - loss: 0.6959 - acc: 0.7854 - val_loss: 0.5556 - val_acc: 0.8450
Epoch 3/8
4217/4217 [==============================] - 1s - loss: 0.4104 - acc: 0.8812 - val_loss: 0.3297 - val_acc: 0.9143
Epoch 4/8
4217/4217 [==============================] - 2s - loss: 0.2998 - acc: 0.9118 - val_loss: 0.2775 - val_acc: 0.9269
Epoch 5/8
4217/4217 [==============================] - 1s - loss: 0.2269 - acc: 0.9386 - val_loss: 0.2516 - val_acc: 0.9293
Epoch 6/8
4217/4217 [==============================] - 1s - loss: 0.1731 - acc: 0.9507 - val_loss: 0.2250 - val_acc: 0.9374
Epoch 7/8
4217/4217 [==============================] - 2s - loss: 0.1480 - acc: 0.9540 - val_loss: 0.2099 - val_acc: 0.9423
Epoch 8/8
4217/4217 [==============================] - 3s - loss: 0.1113 - acc: 0.96